# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools
DirectoryReadTool - allows agent to read any/specific directory \
SerperDevTool - allows you to search internet

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for our team.

Action: Read a file's content
Action Input: {"file_path": "./instructions/DeepLearningAI_background.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: './instructions/DeepLearningAI_background.txt'.
 Tool Read a file's cont

Thought: 
I have gathered enough information about DeepLearningAI's background, key personnel, recent milestones, and other relevant details from the internet search results. It's time to compile this data into a comprehensive report for our team.

Final Answer: 
**Comprehensive Report on DeepLearningAI:**

**Company Background:**
DeepLearningAI is an education technology company founded in 2017 by Andrew Ng, a global leader in AI. The company's mission is to make world-class AI education accessible to people around the globe. DeepLearningAI offers online courses and specializations in artificial intelligence, machine learning, and deep learning.

**Key Personnel:**
- Chief Executive Officer: S. B. R.
- Chief Operating Officer: J. H.
- Chief Operating Officer: S. M. B.
- Chief Strategy Officer (CSO) / SVP of AI: Ted Greenwald
- Professor for Deep Learning and NLP: Manfred Vogel
- Ryan Keenan: Works with Andrew Ng and the team.

**Recent Milestones:**
DeepLearningAI recently celebrated 

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

Subject: Elevating DeepLearningAI's Future with Innovative Solutions

Dear Andrew Ng,

I hope this message finds you well. I wanted to reach out to congratulate you and the DeepLearningAI team on the recent milestone of celebrating the 256th issue of "The Batch." It is truly inspiring to witness the impact that AI education has on individuals worldwide.

As a global leader in AI education, DeepLearningAI has continuously set the bar high in providing expert-led educational experiences for AI practitioners and non-technical professionals. Your commitment to making world-class AI education accessible reflects a shared vision we have at CrewAI.

Our solutions are designed to enhance online learning platforms, improve user engagement, and personalize learning experiences. By streamlining content delivery, optimizing course recommendations, and increasing learner retention, we believe that we can support DeepLearningAI in achieving its goals and fostering a thriving community of learners.

At CrewAI, we are dedicated to offering tailored AI solutions for course development, student assessment, and progress tracking. We understand the importance of personalized engagement strategies in empowering learners to succeed in their AI journey.

I would love the opportunity to discuss how our solutions can align with DeepLearningAI's mission and further elevate your educational initiatives. Please let me know a convenient time for a discussion, and I will be more than happy to accommodate.

Thank you for your time, Andrew. I look forward to the possibility of collaborating with DeepLearningAI and contributing to the future of AI education.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

In [18]:
inputs = {
    "lead_name": "IBM",
    "industry": "IT Company",
    "key_decision_maker": "Arvind Krishna",
    "position": "CEO",
    "milestone": "Watson.data Launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of IBM, a company in the IT Company sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather detailed information about IBM, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. I must use all available data sources to compile a comprehensive profile for effective engagement.

Action: Search the internet
Action Input: {"search_query": "IBM company profile, key decision-makers, recent business developments"} 


Search results: Title: 2023 IBM An



> Entering new CrewAgentExecutor chain...
I need to gather insights on IBM's recent achievements, company culture, and future goals to create a personalized outreach campaign for Arvind Krishna, the CEO.

Action: Search the internet

Action Input: {"search_query": "IBM recent achievements and future goals"}
 


Search results: Title: The best of IBM 2022: 11 stories that matter for the future
Link: https://www.ibm.com/thought-leadership/best-of/2022/
Snippet: This year inspired some truly remarkable innovations. Discover 11 of the best.
---
Title: The best of IBM in 2021: 12 stories of innovation and hope
Link: https://www.ibm.com/thought-leadership/best-of/2021/
Snippet: IBM committed to net-zero gas emissions by 2030​​ In 2021, IBM set goals to get 90% of its energy globally from renewable sources and use technologies like ...
---
Title: Goals and performance - IBM
Link: https://www.ibm.com/about/environment/goals
Snippet: This new goal builds upon IBM's decades of rigorous energy 

In [19]:
from IPython.display import Markdown
Markdown(result)

Subject: Enhancing IBM's Growth with Innovative AI Solutions

Dear Mr. Arvind Krishna,

I hope this message finds you well. I am reaching out to you on behalf of CrewAI with great enthusiasm after learning about IBM's recent achievements and future goals. Your dedication to innovation and growth aligns perfectly with our mission to provide cutting-edge AI solutions that drive efficiency and success.

I was particularly impressed by IBM's commitment to net-zero gas emissions by 2030 and the company's focus on renewable energy sources. At CrewAI, we share your vision for a sustainable future and believe that our AI tools can further enhance IBM's efforts towards environmental responsibility.

Moreover, the breakthrough in quantum computing showcased by IBM is truly commendable and demonstrates the company's ongoing commitment to pushing the boundaries of technology. Our AI tools can complement IBM's quantum computing advancements by streamlining processes, improving decision-making, and driving innovation across various sectors.

I would love the opportunity to discuss how CrewAI's solutions can support IBM's goals in AI adoption, skills development, and sustainable operations. Our team is ready to collaborate with IBM to maximize the potential of your initiatives and drive growth together.

Thank you for considering CrewAI as a partner in your journey towards continued success. I look forward to the possibility of working together and contributing to IBM's future achievements.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI